In [24]:
import math
import numpy as np

# Constants
A0 = 333  # Administered activity (MBq)
T = 1  # Occupancy factor1
T1_2 = 109.8 / 60  # Radionuclide half-life in hours (109.8 minutes)
dr = 0.092  # Dose rate immediately after administration (µSv m²/MBq h)
P_controlled = 100  # Weekly dose limit for controlled areas (µSv)
P_uncontrolled = 20  # Weekly dose limit for uncontrolled areas (µSv)

# Function to get user input for variables
def get_inputs():
    Nw = int(input("Enter Number of patients per week: "))
    d = float(input("Enter Distance from source to barrier (m): "))
    tU = float(input("Enter Uptake time (hours): "))
    tI = float(input("Enter Scan time (hours): "))
    print(f"Input values - Nw: {Nw}, d: {d}, tU: {tU}, tI: {tI}")
    return Nw, d, tU, tI



# Function to calculate the dose reduction factor over imaging time
def RtI(T1_2, tI):
    rti_value = 1.443 * (T1_2 / tI) * (1 - math.exp(-0.693 * tI / T1_2))
    print(f"RtI: {rti_value}")
    return rti_value



def Fu(tU):
    return math.exp (-0.693 * (tU*60) /110)

# Function to calculate transmission factor for the Scan room
def B_scan_room(P, d, T, Nw, A0, tI, Fu, RtI):
    return 12.8 * P * (d ** 2) / (T * Nw * A0 * 0.85 * tI * Fu* RtI)




# Main function to run the calculation for both controlled and uncontrolled areas
def calculate_transmission_factors():
    # Get user inputs
    Nw, d, tU, tI = get_inputs()
    
    # Calculate the dose reduction factor RtU & RtI
    rti = RtI(T1_2, tI)

    # Calculate the Fu
    fu = Fu(tU)

    # Calculate the transmission factor B for uncontrolled and controlled areas
    B_S_uncontrolled = B_scan_room(P_uncontrolled,d, T, Nw, A0, tI, fu, rti)
    B_S_controlled = B_scan_room(P_controlled, d, T, Nw, A0, tI, fu, rti)

    
     # Output the results
    print()  # This adds a blank line before the following prints
    print("****SCAN ROOM****")
    print(f"\nTransmission Factor (B) for the Scan room (Uncontrolled area): {B_S_uncontrolled:.4f}")
    print(f"Transmission Factor (B) for the Scan room (Controlled area): {B_S_controlled:.4f} ")

    return {

        "Scan Room Uncontrolled": B_S_uncontrolled,
        "Scan Room Controlled": B_S_controlled
    }

# Run the calculations and store the results
B_results = calculate_transmission_factors()



# Function to calculate lead and concrete thickness based on the transmission factor B
def calculate_thickness(B_results):
    """
    Calculate thickness for both lead and concrete based on the transmission factor B.
    Lead: d = log10(1 / B) * 16.6 mm
    Concrete: d = log10(1 / B) * 17.6 cm
    
    Parameters:
    B_results: Dictionary containing transmission factors for uncontrolled and controlled areas
    
    Output:
    Lead thickness in millimeters and Concrete thickness in centimeters
    """
    # Extract B values from the dictionary
    B_uncontrolled = B_results["Scan Room Uncontrolled"]
    B_controlled = B_results["Scan Room Controlled"]

    # Calculate lead and concrete thickness for uncontrolled area
    lead_thickness_uncontrolled = np.log10(1 / B_uncontrolled) * 16.6  # Lead thickness in mm
    concrete_thickness_uncontrolled = np.log10(1 / B_uncontrolled) * 17.6  # Concrete thickness in cm

    # Calculate lead and concrete thickness for controlled area
    lead_thickness_controlled = np.log10(1 / B_controlled) * 16.6  # Lead thickness in mm
    concrete_thickness_controlled = np.log10(1 / B_controlled) * 17.6  # Concrete thickness in cm
    
    return (lead_thickness_uncontrolled, concrete_thickness_uncontrolled), (lead_thickness_controlled, concrete_thickness_controlled)

# Calculate thickness for both lead and concrete using the obtained B value
(thickness_uncontrolled, thickness_controlled) = calculate_thickness(B_results)

# Output the calculated thicknesses

print()  # This adds a blank line before the following prints
print("\033[1mUnControlled Area:\033[0m")
print(f"Lead thickness: {thickness_uncontrolled[0]:.2f} mm")
print(f"Concrete thickness: {thickness_uncontrolled[1]:.2f} cm")
print()
print("\033[1mControlled Area:\033[0m")
print(f"Lead thickness: {thickness_controlled[0]:.2f} mm")
print(f"Concrete thickness: {thickness_controlled[1]:.2f} cm")

Input values - Nw: 48, d: 1.0, tU: 1.0, tI: 0.5
RtI: 0.9110297051044933

****SCAN ROOM****

Transmission Factor (B) for the Scan room (Uncontrolled area): 0.0604
Transmission Factor (B) for the Scan room (Controlled area): 0.3018 

UnControlled Area:
Lead thickness: 20.24 mm
Concrete thickness: 21.46 cm

Controlled Area:
Lead thickness: 8.64 mm
Concrete thickness: 9.16 cm
